In [1]:
import logging
import uuid
import sys
from pathlib import Path

import cv2
import depthai as dai # access the camera and its data packets
#from imutils.video import FPS

In [2]:
log = logging.getLogger(__name__)

In [3]:
model_name= "mobilenet-ssd_openvino_2021.2_6shave.blob"

In [4]:
models_folder = Path.cwd().parent.joinpath('models')
nn_path = str(Path(models_folder).joinpath(model_name))
print(nn_path)

h:\openCV-museum-guards\models\mobilenet-ssd_openvino_2021.2_6shave.blob


In [5]:
# Start defining a pipeline
pipeline = dai.Pipeline()

In [6]:
# Define a source - color camera
cam_rgb = pipeline.createColorCamera()
cam_rgb.setPreviewSize(300, 300)
cam_rgb.setBoardSocket(dai.CameraBoardSocket.RGB)
cam_rgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam_rgb.setInterleaved(False)
cam_rgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.RGB)

# Create output
xout_rgb = pipeline.createXLinkOut()
xout_rgb.setStreamName("rgb_preview_out")
cam_rgb.preview.link(xout_rgb.input)

In [9]:
# Define a source - two mono (grayscale) cameras
left = pipeline.createMonoCamera()
left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
left.setBoardSocket(dai.CameraBoardSocket.LEFT)

right = pipeline.createMonoCamera()
right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
right.setBoardSocket(dai.CameraBoardSocket.RIGHT)

# Create a node that will produce the depth map (using disparity output as it's easier to visualize depth this way) # TODO: understand why disparity output is easier to visualize
depth = pipeline.createStereoDepth()
depth.setConfidenceThreshold(200) # TODO:  what are the different values for this threshold?
left.out.link(depth.left)
right.out.link(depth.right)

# Create output for disparity
xout = pipeline.createXLinkOut()
xout.setStreamName("disparity")
depth.disparity.link(xout.input)

# Create output for depth
depthout = pipeline.createXLinkOut()
depthout.setStreamName("depth")
depth.depth.link(depthout.input)

In [7]:
# Define a neural network that will make predictions based on the source frames
nn = pipeline.createMobileNetDetectionNetwork()
nn.setConfidenceThreshold(0.5) # TODO: create variable
nn.setBlobPath(nn_path)
nn.setNumInferenceThreads(2) # TODO: learn more about the number of inference threads
nn.input.setBlocking(False) # TODO: learn about setBlocking True vs. False

In [13]:
# Create a node to convert the grayscale frame into the nn-acceptable form
manip = pipeline.createImageManip()
manip.initialConfig.setResize(300, 300)
# The NN model expects BGR input. By default ImageManip output type would be same as input (gray in this case)
manip.initialConfig.setFrameType(dai.RawImgFrame.Type.BGR888p)
right.out.link(manip.inputImage)
manip.out.link(nn.input)

# Create outputs
manipOut = pipeline.createXLinkOut()
manipOut.setStreamName("NN")
manip.out.link(manipOut.input)


In [11]:
#output = pipeline.getAllAssets()

In [14]:
# Pipeline defined, now the device is connected to
with dai.Device(pipeline) as device:
    # Start pipeline
    device.startPipeline()

    # Output queues will be used to get the grayscale / depth frames and nn data from the outputs defined above
    rgb = device.getOutputQueue("rgb_preview_out", maxSize=4, blocking=False)
    qDisparity = device.getOutputQueue("disparity", maxSize=4, blocking=False)
    qDepth = device.getOutputQueue("depth", maxSize=4, blocking=False)

    rgbFrame = None
    disparityFrame = None

    while True:
        inRight = rgb.get()
        inDisparity = qDisparity.get()

        rightFrame = inRight.getFrame()
        rightFrame = cv2.flip(rightFrame, flipCode=1)
        cv2.imshow("rectified right", rightFrame)


        #disparityFrame = inDisparity.getFrame()
        #cv2.imshow("disparity", disparityFrame)

        if cv2.waitKey(1) == ord('q'):
            cv2.destroyAllWindows()  # needed to avoid openCV from crashing
            break

error: OpenCV(4.5.1) c:\users\appveyor\appdata\local\temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x206ccf44::Set<3,4,-1>,struct cv::impl::A0x206ccf44::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 300
